In [1]:
import time
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os

import warnings
warnings.filterwarnings(action='ignore')

data_path = '/opt/ml/input/data/train'

In [2]:
train_df = pd.read_csv(os.path.join(data_path, 'train_ratings.csv')) # 전체 학습 데이터
year_data = pd.read_csv(os.path.join(data_path, 'years.tsv'), sep='\t')
writer_data = pd.read_csv(os.path.join(data_path, 'writers.tsv'), sep='\t')
title_data = pd.read_csv(os.path.join(data_path, 'titles.tsv'), sep='\t')
genre_data = pd.read_csv(os.path.join(data_path, 'genres.tsv'), sep='\t')
director_data = pd.read_csv(os.path.join(data_path, 'directors.tsv'), sep='\t')

# 유저 프로파일

In [3]:
# 2005 년에 본 영화의 경우 주로 몇넌도 개봉 영화인지 파악


def get_item_in_year(train_df, title_data, year):
    '''
    2005년 까지의 영화 시청 기록안에는 미래의 영화 시청 기록이 없음
    따라서 user가 주로 소비한 영화의 시청 연도는 영화를 필터링 하는 것에 중요한 요소로 작용할 수 있음
    ex) 2005년 주로 영화를 시청한 유저는 2005년 이후에 개봉한 영화를 추천해주는 것은 사실상 의미가 없음 (근데 이건 본 대회 Task에만 적용하는 문제임) 
    '''
    item_list = train_df[train_df['year'] <= year]['item'].unique().tolist()
    item_list = title_data.set_index('item').loc[item_list, :]['year'].unique().tolist()
    print(f'{year} 년')
    print(f'최대 년도: {max(item_list)}')
    print(f'최소 년도: {min(item_list)}')

In [4]:
import re

# year data에 결측치가 존재하여 따로 title에서 year를 가져옴

def get_title_year(x):
    # 가로 안에 있는 4자리의 숫자만 매칭
    pattern = r"\(([\d]{4})\)"
    ret = re.search(pattern, x)
    if ret: return ret.group(1)
    else: 
        if x == 'Fawlty Towers (1975-1979)': return '1975'
        if x == 'Big Bang Theory, The (2007-)': return '2007'

title_data['year'] = title_data['title'].apply(get_title_year)

In [7]:
title_data.to_csv(os.path.join(data_path, 'item_pro.csv'), index = False)

In [21]:
title_data

,item,title,year
0,318,"Shawshank Redemption, The (1994)",1994
1,2571,"Matrix, The (1999)",1999
2,2959,Fight Club (1999),1999
3,296,Pulp Fiction (1994),1994
4,356,Forrest Gump (1994),1994
...,...,...,...
6802,73106,American Pie Presents: The Book of Love (Ameri...,2009
6803,109850,Need for Speed (2014),2014
6804,8605,Taxi 3 (2003),2003
6805,3689,Porky's II: The Next Day (1983),1983


# 아이템 프로파일

In [18]:
def get_item_in_year(user, user_item_list, title_data):
    user_item_year_list = title_data.set_index('item').loc[user_item_list, :]['year'].unique().tolist()
    min_year = min(user_item_year_list)
    max_year = max(user_item_year_list)
    return min_year, max_year

In [19]:
group_df = train_df.groupby('user')
user_data = []
for user, df in group_df:
    user_item_list = df['item'].tolist()
    min_year, max_year = get_item_in_year(user, user_item_list, title_data)
    user_data.append({
        'user' : user,
        'min_year' : min_year, 
        'max_year' : max_year, 
    })

user_data = pd.DataFrame(user_data)
user_data

,user,min_year,max_year
0,11,1941,2009
1,14,1925,2008
2,18,1952,2009
3,25,1939,2008
4,31,1974,2014
...,...,...,...
31355,138473,1932,2010
31356,138475,1920,2005
31357,138486,1939,2007
31358,138492,1959,2004


In [20]:
user_data.to_csv(os.path.join(data_path, 'user_pro.csv'), index = False)